# Practical on Genome-Wide Association Studies
## Tuesday afternoon, 21/6/2022

#### What's included in this directory
* `example_1/` containing `data/` with `example_1.[bed, bim, fam]` and `pheno_1.pheno` files along with information on structure/stratification.
* `example_2/` containing `data/` with `example_2.[bed, bim, fam]` and `pheno_2.pheno` files.
* `example_3/` containing `data/` with `example_3.[bed, bim, fam]` and `pheno_3.pheno` files. 

#### The following two blocks contain paths to the programmes we will use and useful plotting functions. Feel free to scroll down to Exercise 1.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import dash_bio
from scipy.stats import chi2
import matplotlib.pyplot as plt
from IPython.display import Image, display
import seaborn as sns

# Hardlinks to the software we need:
PLINK="/opt/conda/bin/plink2"
BOLT="/usr/local/bin/bolt"
REGENIE="/usr/local/bin/regenie"

In [ ]:
def draw_manhattan_plot(sumstats, adjusted_sumstats=None):
    ss = pd.read_table(sumstats, delim_whitespace=True)
    if "bolt" in sumstats:
        ss['P'] = ss.P_BOLT_LMM_INF
        ss['GENE'] = ""
    else:
        ss = ss[ss.TEST == 'ADD']

    if ss.columns[0] == "#CHROM":
        ss = ss.rename(columns = {
            '#CHROM': 'CHR',
            'POS': 'BP'
        })
        ss['GENE'] = ""
        ss['SNP'] = ss['BP']
    if adjusted_sumstats is not None:
        adf = pd.read_table(adjusted_sumstats, delim_whitespace=True)
        adf = adf.rename(columns = {'#CHROM': 'CHR'})
        df = pd.merge(adf, ss, on=['ID', 'CHR'])
        df['P'] = df.GC
        ss = df.sort_values(['CHR', 'BP'])

    out_path = sumstats.split("/")[-1].replace("glm.linear", "manhattan.png")
    fig = dash_bio.ManhattanPlot(dataframe=ss, title="", showlegend=False, highlight=False, genomewideline_value=-np.log10(5e-8))
    out_path = f"{sumstats}.manhattan.png"
    print(f"Writing to {out_path}")
    fig.write_image(out_path, width=600, height=350)
    display(Image(out_path))
    
def draw_qq_plot(sumstats):

    ss = pd.read_table(sumstats, delim_whitespace=True)
    if "bolt" in sumstats:
        pvals = ss.P_BOLT_LMM_INF
    elif "adjusted" in sumstats:
        pvals = ss.GC
    else:
        ss = ss[ss.TEST == 'ADD']
        pvals = ss.P
    sorted_pvals = np.sort(pvals)

    # Expected P-values are uniformly distributed
    pval_grid = np.linspace(0, 1, len(sorted_pvals) + 1)
    expected_pvals = (pval_grid[:-1] + pval_grid[1:]) / 2

    # Calculate genomic control factor
    p_median = np.nanmedian(sorted_pvals)
    lambda_gc = chi2.ppf(1 - p_median, df=1) / 0.456

    # Set up a scatter plot of the results
    out_path = sumstats.split("/")[-1] + ".qq.png"
    _, ax = plt.subplots()
    plt.scatter(x=-np.log10(expected_pvals), y=-np.log10(sorted_pvals), s=3)
    plt.xlabel("Expected -log10 P-values")
    plt.ylabel("Observed -log10 P-values")
    plt.title("QQ-plot of association statistics")
    plt.axline((0, 0), slope=1, color='red')
    plt.text(0.1, 0.8, f"Genomic control factor = {lambda_gc:.3f}", transform = ax.transAxes)
    plt.show()

def draw_top_2_pcs(eigenvec, pops=None):
    vec_table = pd.read_table(eigenvec, delim_whitespace=True)

    if pops is not None and os.path.exists(pops):
        pop_table = pd.read_table(pops, delim_whitespace=True)
        table = pd.merge(vec_table, pop_table, on="IID")
        sns.scatterplot(data=table, x="PC1", y="PC2", hue='POP').set(title="Principal components")
    else:
        sns.scatterplot(data=vec_table, x="PC1", y="PC2").set(title="Principal components")
    plt.show()

def draw_stratification(phenotype, populations):
    pheno_table = pd.read_table(phenotype, delim_whitespace=True)
    pop_table = pd.read_table(populations, delim_whitespace=True)
    table = pd.merge(pheno_table, pop_table, on='IID')

    sns.violinplot(data=table, x='POP', y="pheno_1").set(title="Phenotype distribution for each population")
    plt.show()

def draw_relatedness(rel_path):
    rel_cut_off = 0.25

    max_rel = []
    with open(rel_path, 'r') as relfile:
        for i, line in enumerate(relfile):
            l = [float(r) for r in line.strip().split()]
            del l[i]
            max_rel.append(max(l))

    plt.figure(figsize=(6,5))
    sns.boxplot(data=max_rel, whis=np.inf)
    ax = sns.stripplot(data=max_rel, color="black", edgecolor="gray", size=3)
    ax.axhline(0.25, color='black', linestyle='--')
    ax.axes.xaxis.set_visible(False)
    plt.title("Maximum relatedness per sample")
    plt.ylim(0, 1)
    plt.show()

def draw_multiple_qq():
    pvals = {}
    sumstats="example_2/data/gwas.linreg.pheno_2.glm.linear"
    ss = pd.read_table(sumstats, delim_whitespace=True)
    ss = ss[ss.TEST == 'ADD']
    pvals["LR naive"]  = ss.P

    sumstats="example_2/data/gwas_adjust.pheno_2.glm.linear.adjusted"
    ss = pd.read_table(sumstats, delim_whitespace=True)
    pvals["LR + GC"] = ss.GC

    sumstats="example_2/data/gwas.bolt.pheno_2.sumstats.gz"
    ss = pd.read_table(sumstats, delim_whitespace=True)
    pvals["BOLT:Inf"] = ss.P_BOLT_LMM_INF

    sumstats="example_2/data/gwas.regenie_pheno_2.regenie.gz"
    ss = pd.read_table(sumstats, delim_whitespace=True)
    pvals["Regenie"] = 10**(-ss.LOG10P)

    _, ax = plt.subplots(figsize=(6,6))

    for met in pvals:
        sorted_pvals = np.sort(pvals[met])

        # Expected P-values are uniformly distributed
        pval_grid = np.linspace(0, 1, len(sorted_pvals) + 1)
        expected_pvals = (pval_grid[:-1] + pval_grid[1:]) / 2

        # Calculate genomic control factor
        p_median = np.nanmedian(sorted_pvals)
        lambda_gc = chi2.ppf(1 - p_median, df=1) / 0.456
        # Set up a scatter plot of the results
        plt.scatter(x=-np.log10(expected_pvals), y=-np.log10(sorted_pvals), s=3, label=f"{met} | {lambda_gc:.3f}")

    X=-np.log10(expected_pvals)
    plt.plot( [min(X),max(X)], [min(X),max(X)], color='grey', linestyle='--')
    plt.xlabel("Expected -log10 P-values")
    plt.ylabel("Observed -log10 P-values")
    plt.title("QQ-plot of association statistics")
    plt.legend(title="Method   |   $\lambda_{GC}$")
    plt.show()

# Exercise 1
We have simulated 10 short chromosomes for 1,000 individuals and generated a heritable phenotype in `example_1/data/example_1.[bim/bed/fam]`. In this exercise we will perform a GWAS and examine structure in the sample.

We start by running a GWAS using the simple linear model $y = bx + \epsilon$.

In [ ]:
!{PLINK} \
    --bfile example_1/data/example_1 \
    --pheno example_1/data/pheno_1.pheno \
    --glm \
    --out example_1/data/gwas.linreg

This prints GWAS summary statistics to `example_1/data/gwas.linreg.pheno_1.glm.linear`.

We can explore the results using the following plotting functions.

In [ ]:
sumstats_path = "example_1/data/gwas.linreg.pheno_1.glm.linear"

draw_manhattan_plot(sumstats_path)

draw_qq_plot(sumstats_path)

#### The summary statistics are considerably inflated.
* What could be the issue and how could we verify our hypotheses?
* How can we proceed and remove the confounding?

We start by performing PCA on the genotypes. We extract the top 10 principal components using all markers with allele frequency >5% and draw up the top 2 principal components.

In [ ]:
!{PLINK} \
    --bfile example_1/data/example_1 \
    --pca 10 \
    --maf 0.05 \
    --out example_1/data/example_1

draw_top_2_pcs("example_1/data/example_1.eigenvec", "example_1/data/populations_1.txt")

#### Our sample looks like it has considerable structure.
To adjust for population structure and stratification, we add the top 10 principal components as covariates to our model: $y = bx + \text{PC}_1\alpha_1 + ... \text{PC}_{10}\alpha_{10} + \epsilon$.

In [ ]:
!{PLINK} \
    --bfile example_1/data/example_1 \
    --pheno example_1/data/pheno_1.pheno \
    --glm \
    --covar-variance-standardize \
    --covar example_1/data/example_1.eigenvec \
    --out example_1/data/gwas.pca

draw_manhattan_plot("example_1/data/gwas.pca.pheno_1.glm.linear")

draw_qq_plot("example_1/data/gwas.pca.pheno_1.glm.linear")

#### Why did we observe all this inflation? 
If we look under the hood of the simulation, we see that the underlying demographic model has three distinct populations. This is just as we expected from the principal component analysis. There is also considerable stratification in the phenotype.

In [ ]:
draw_stratification('example_1/data/pheno_1.pheno', 'example_1/data/populations_1.txt')
display(Image("example_1/data/demes.png"))

# Exercise 2

In this exercise, we proceed as in the previous one. We have data of the same shape and size saved in `example_2/data/example_2.[bim/bed/fam]` and `example_2/data/pheno_2.pheno`, and start with a linear regression GWAS.

In [ ]:
!{PLINK} \
    --bfile example_2/data/example_2 \
    --pheno example_2/data/pheno_2.pheno \
    --glm \
    --out example_2/data/gwas.linreg

In [ ]:
sumstats_path = "example_2/data/gwas.linreg.pheno_2.glm.linear"

draw_manhattan_plot(sumstats_path)

draw_qq_plot(sumstats_path)

Again we observe inflation, but in this case the principal components offer little insight:

In [ ]:
!{PLINK} \
    --bfile example_2/data/example_2 \
    --pca 10 \
    --maf 0.05 \
    --out example_2/data/example_2

draw_top_2_pcs("example_2/data/example_2.eigenvec")

#### 
With $\lambda_{GC}\gg 1$ we are likely to find many false positives in our study. Principal components look like they will not help this time.

A crude way of controlling for inflation is applying genomic control.
 
Recall that $\lambda_{GC} = \frac{ \text{observed median} \chi^2}{ \text{expected median} \chi^2}$. To apply genomic control, we divide each $\chi^2_i$ with $\lambda_{GC}$.

We run plink again using the `--adjust` command. In addition to the linear regression GWAS, this prints adjusted P-values to `example_2/data/gwas_adjust.pheno_2.glm.linear.adjusted`.

In [ ]:
!{PLINK}  \
    --bfile example_2/data/example_2 \
    --pheno example_2/data/pheno_2.pheno \
    --glm \
    --adjust \
    --out example_2/data/gwas_adjust

draw_qq_plot("example_2/data/gwas_adjust.pheno_2.glm.linear.adjusted")
draw_manhattan_plot("example_2/data/gwas_adjust.pheno_2.glm.linear", "example_2/data/gwas_adjust.pheno_2.glm.linear.adjusted")

It looks like genomic control is too conservative in our setting as the lowest p-value is now less than $5\cdot 10^{-6}$.

But we have one more trick up our sleeve, the linear mixed model $y = X\beta + Gu + \epsilon$.
We run the LMM using BOLT-LMM and see if we get an improvement.

In [ ]:
!{BOLT} \
    --bfile=example_2/data/example_2 \
    --phenoFile=example_2/data/pheno_2.pheno \
    --phenoCol pheno_2 \
    --verboseStats \
    --statsFile=example_2/data/gwas.bolt.pheno_2.sumstats.gz \
    --lmmInfOnly

draw_qq_plot("example_2/data/gwas.bolt.pheno_2.sumstats.gz")
draw_manhattan_plot("example_2/data/gwas.bolt.pheno_2.sumstats.gz")

Recall that the LMM algorithm has two steps: Model fitting and association. Briefly read through the BOLT-LMM logs and see if you can recognise them.
* What is the heritability that BOLT estimates?
* What is the calibration factor?
* BOLT-LMM gives us $\lambda_{GC}$ both for the LMM and linear regression. Note the difference.


#### Why were our summary statistics inflated?
If not population structure, the other usual suspect is relatedness. One heuristic way of finding related samples is looking at the maximum value per row in the GRM $A$. If $A_{ij}>0.25$, we can conclude that individuals $i$ and $j$ are probably related.

If we plot the closest kinship for each individual, we see that our samples are extremely related.

In [ ]:
!{PLINK} \
    --bfile example_2/data/example_2 \
    --make-rel square \
    --out example_2/data/example_2

draw_relatedness("example_2/data/example_2.rel")

Another popular program to run an LMM is Regenie. Regenie is less accurate than BOLT but can scale to larger sample sizes. Running Regenie has two steps:

In [ ]:
!{REGENIE} \
    --step 1 --bsize 500 \
    --bed example_2/data/example_2 \
    --phenoFile example_2/data/pheno_2.pheno \
    --out example_2/data/gwas.regenie \
    --gz

!{REGENIE} \
    --step 2 --bsize 1000\
    --bed example_2/data/example_2 \
    --phenoFile example_2/data/pheno_2.pheno \
    --minMAC 1 \
    --pred example_2/data/gwas.regenie_pred.list \
    --out example_2/data/gwas.regenie \
    --gz

The following block of code compares the inflation of linear regression, linear regression with genomic control, BOLT-LMM and Regenie.

In [ ]:
draw_multiple_qq()

# Exercise 3
In this exercise we provide data in  `example_3/data/example_3.[bim/bed/fam]` and `example_3/data/pheno_3.pheno` but leave the analysis to you.

By going through the following steps 

#### 1. Perform a GWAS using linear regression and draw a Manhattan plot

In [ ]:
# Code goes here

#### 2. Draw a QQ plot of the results

In [ ]:
# Code goes here

#### 3. Perform PCA and inspect the top 2 principal components for structure

In [ ]:
# Code goes here

#### 4. Build the GRM and inspect it for related samples

In [ ]:
# Code goes here

You should now have enough information to draw conclusions about the structure of the sample and the phenotype. Was there inflation? Was it caused by structure, relatedness, or something else entirely?

#### 5. For extra credit, see if you can run BOLT-LMM on the phenotype and inspect the output. Does this give extra insight?

In [ ]:
# Code goes here